# Text Generation using RNN

## Import

In [21]:
import numpy as np
import pandas as pd

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

## RNN을 이용하여 텍스트 생성하기

### Text Preprocessing

In [80]:
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n
"""

In [81]:
# 단어 집합을 생성
t = Tokenizer()
t.fit_on_texts([text])

In [82]:
# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야하므로 미리 +1 선언
vocab_size = len(t.word_index) + 1 
print("Num of words", vocab_size)

Num of words 12


In [83]:
t.word_index

{'말이': 1,
 '경마장에': 2,
 '있는': 3,
 '뛰고': 4,
 '있다': 5,
 '그의': 6,
 '법이다': 7,
 '가는': 8,
 '고와야': 9,
 '오는': 10,
 '곱다': 11}

In [84]:
# 훈련 데이터 만들기
sequences = list()
for line in text.split("\n"):
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
print("학습에 사용할 샘플 수", len(sequences))

학습에 사용할 샘플 수 11


In [85]:
sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [86]:
max_len = max([len(i) for i in sequences])
print("샘플의 최대 길이:", max_len)

샘플의 최대 길이: 6


In [87]:
# add padding
## padding의 인자로 'pre'를 주면 길이가 6보다 짧은 샘플의 앞에 0으로 채웁
sequences = pad_sequences(sequences, maxlen=max_len, padding="pre")
sequences

array([[ 0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  2,  3,  1],
       [ 0,  0,  2,  3,  1,  4],
       [ 0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  6,  1],
       [ 0,  0,  0,  6,  1,  7],
       [ 0,  0,  0,  0,  8,  1],
       [ 0,  0,  0,  8,  1,  9],
       [ 0,  0,  8,  1,  9, 10],
       [ 0,  8,  1,  9, 10,  1],
       [ 8,  1,  9, 10,  1, 11]], dtype=int32)

In [88]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]
X.shape, y.shape

((11, 5), (11,))

In [89]:
y

array([ 3,  1,  4,  5,  1,  7,  1,  9, 10,  1, 11], dtype=int32)

In [90]:
y = to_categorical(y, num_classes=vocab_size)
y.shape

(11, 12)

In [91]:
y

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

### Build RNN model

In [92]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation="softmax"))
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 5, 10)             120       
_________________________________________________________________
simple_rnn_7 (SimpleRNN)     (None, 32)                1376      
_________________________________________________________________
dense_7 (Dense)              (None, 12)                396       
Total params: 1,892
Trainable params: 1,892
Non-trainable params: 0
_________________________________________________________________


In [93]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [94]:
model.fit(X, y, epochs=200, verbose=2)

Train on 11 samples
Epoch 1/200
11/11 - 1s - loss: 2.4751 - accuracy: 0.2727
Epoch 2/200
11/11 - 0s - loss: 2.4610 - accuracy: 0.2727
Epoch 3/200
11/11 - 0s - loss: 2.4470 - accuracy: 0.3636
Epoch 4/200
11/11 - 0s - loss: 2.4330 - accuracy: 0.3636
Epoch 5/200
11/11 - 0s - loss: 2.4188 - accuracy: 0.3636
Epoch 6/200
11/11 - 0s - loss: 2.4043 - accuracy: 0.2727
Epoch 7/200
11/11 - 0s - loss: 2.3896 - accuracy: 0.3636
Epoch 8/200
11/11 - 0s - loss: 2.3745 - accuracy: 0.3636
Epoch 9/200
11/11 - 0s - loss: 2.3590 - accuracy: 0.3636
Epoch 10/200
11/11 - 0s - loss: 2.3431 - accuracy: 0.3636
Epoch 11/200
11/11 - 0s - loss: 2.3266 - accuracy: 0.3636
Epoch 12/200
11/11 - 0s - loss: 2.3095 - accuracy: 0.3636
Epoch 13/200
11/11 - 0s - loss: 2.2918 - accuracy: 0.3636
Epoch 14/200
11/11 - 0s - loss: 2.2735 - accuracy: 0.3636
Epoch 15/200
11/11 - 0s - loss: 2.2545 - accuracy: 0.3636
Epoch 16/200
11/11 - 0s - loss: 2.2350 - accuracy: 0.3636
Epoch 17/200
11/11 - 0s - loss: 2.2149 - accuracy: 0.3636
Epo

Epoch 142/200
11/11 - 0s - loss: 0.2562 - accuracy: 1.0000
Epoch 143/200
11/11 - 0s - loss: 0.2510 - accuracy: 1.0000
Epoch 144/200
11/11 - 0s - loss: 0.2458 - accuracy: 1.0000
Epoch 145/200
11/11 - 0s - loss: 0.2407 - accuracy: 1.0000
Epoch 146/200
11/11 - 0s - loss: 0.2358 - accuracy: 1.0000
Epoch 147/200
11/11 - 0s - loss: 0.2310 - accuracy: 1.0000
Epoch 148/200
11/11 - 0s - loss: 0.2262 - accuracy: 1.0000
Epoch 149/200
11/11 - 0s - loss: 0.2216 - accuracy: 1.0000
Epoch 150/200
11/11 - 0s - loss: 0.2171 - accuracy: 1.0000
Epoch 151/200
11/11 - 0s - loss: 0.2127 - accuracy: 1.0000
Epoch 152/200
11/11 - 0s - loss: 0.2083 - accuracy: 1.0000
Epoch 153/200
11/11 - 0s - loss: 0.2041 - accuracy: 1.0000
Epoch 154/200
11/11 - 0s - loss: 0.2000 - accuracy: 1.0000
Epoch 155/200
11/11 - 0s - loss: 0.1959 - accuracy: 1.0000
Epoch 156/200
11/11 - 0s - loss: 0.1920 - accuracy: 1.0000
Epoch 157/200
11/11 - 0s - loss: 0.1881 - accuracy: 1.0000
Epoch 158/200
11/11 - 0s - loss: 0.1843 - accuracy: 1.00

### Predict

In [95]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [96]:
sentence_generation(model, t, "경마장에", 4)

'경마장에 있는 말이 뛰고 있다'

In [97]:
sentence_generation(model, t, "그의", 4)

'그의 말이 법이다 오는 말이'

In [98]:
sentence_generation(model, t, "말이", 4)

'말이 말이 고와야 오는 말이'